In [ ]:
%matplotlib notebook
%load_ext autoreload

# Analysis

Notebook containing code to create our plots

In [ ]:
import pandas as pd
from tqdm.autonotebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import pickle
from typing import Dict


plt.style.use(['science', 'notebook'])

## Preambula
We are going to run some computations, to save time let's create a decorator that stores and read from disk

In [ ]:
def maybe_load_from_disk(location: Path):
    def decorator(func):
        def _inner(*args, **kwargs):
            if location.exists():
                print(f"[INFO] loading from {location}")
                with open(location, "rb") as f:
                    return pickle.load(f)
            res = func(*args, **kwargs)
            with open(location, "wb") as f:
                print(f"[INFO] saving to {location}")
                pickle.dump(res, f)
            return res
        return _inner
    return decorator
    

In [ ]:
df = pd.read_csv("../metadata/categories.csv", index_col=0)

df.head()

# Sizes

Find out total dataset sizes, we have `rf100` download at `rf100`. We can use the index to iterate and get the size of each folder

In [ ]:
from pathlib import Path
from functools import reduce
from collections import defaultdict

RF100_ROOT = Path('../rf100')

def count_num_files(dataset: str):
    dataset_path = RF100_ROOT / dataset
    sub_dirs = ["train", "valid", "test"]
    num_files = defaultdict(int)
    for sub_dir in sub_dirs:
        sub_dir_path = dataset_path / sub_dir / 'images'
        num_files[sub_dir] += sum([1 if curr.is_file() else 0 for curr in sub_dir_path.iterdir()])
    
    return pd.Series(num_files)

In [ ]:
# @maybe_load_from_disk(Path('../temp/df.pkl'))
def apply_num_files(df):
    df[["train", "test", "valid"]] = df.apply(lambda row: count_num_files(row.name), axis=1)[["train", "test", "valid"]]
    df["size"] = df["train"] +  df["test"] +  df["valid"]
    return df

df = apply_num_files(df)

We now want to add the number of classes for each dataset, obtained before hand

In [ ]:
import json
# obtained by running `export ROBOFLOW_API_KEY=.... && python ./scripts/get_labels_names.py` 
with open("../metadata/labels_names.json", 'r') as f: 
    labels_names = json.load(f)

In [ ]:
df

In [ ]:
def get_num_classes_per_dataset(labels_names: Dict) -> pd.DataFrame:
    records = []
    for item in labels_names:
        num_classes = len(item["classes"])
        records.append({
            "dataset" : item["name"],
            "num_classes": num_classes
        })
    return pd.DataFrame.from_records(records).set_index("dataset")

df = df.join(get_num_classes_per_dataset(labels_names))

Finally, we also want to add the yolov5/7 - glip results

In [ ]:
results = pd.read_csv("../results.csv", index_col=0)
df = df.join(results)

In [ ]:
df

Let's see how many of them there are for each category

In [ ]:
df

In [ ]:
df["num_datasets"] = 1
aggretations = {
    "train" : "sum", "test" : "sum", "valid" : "sum", "size" : "sum", "num_classes" : "sum",
    "yolov5": "mean", "yolov7": "mean",
    "num_datasets" : "sum"           
                }
grouped_df = df.groupby("category").agg(aggretations).reset_index()
grouped_df = grouped_df.sort_values("size")
grouped_df["perc"] = grouped_df["size"] / grouped_df["size"].sum()
grouped_df

Now, we want to use the order of the categories to sort our original dataframe, till will make it easier to visualize them

In [ ]:
df_with_ordered_categories = pd.DataFrame(index=grouped_df.index, data={"category": grouped_df.category})
df = df_with_ordered_categories.merge(df.reset_index("dataset"), on="category", how="inner")
df = df.set_index("dataset")
df

Let's store it to disk

In [ ]:
df.to_csv("../metadata/datasets_stats.csv")

## Bounding boxes stats

Cool, so we may also want to plot/show the mean size of bboxes for each dataset

Let's create something to read all the annotations. We can take advantage of PyTorch Dataloader to use multiple cores and make the computation go brum brum

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch

IGNORE = -1
# all images are resized to 640
size = (640, 640)

class AnnotationDataset(Dataset):
    def __init__(self, root: Path, fmt: str = "txt"):
        super().__init__()
        self.annotations_paths = list(root.glob(f"**/*.{fmt}"))
    
    def maybe_convert_polygon_to_bbox(self, line: str):
        splitted = line.split(" ")
        label, rest = splitted[0], splitted[1:]
        label = torch.as_tensor(int(label))
        is_bbox = len(rest) == 4
        if is_bbox:
            return  label, torch.as_tensor([float(el) for el in rest])
        else:
            # must be a polygon
            poly = torch.as_tensor([float(el) for el in rest])
            poly = poly.view(-1, 2)
            xmax, ymax = torch.max(poly, dim=0).values
            xmin, ymin = torch.min(poly, dim=0).values
            width, heigh = xmax - xmin, ymax - ymin
            xcenter, ycenter =  xmin + width / 2, ymin + heigh / 2
            return label, torch.stack([xcenter, ycenter, width, heigh])
            
    def __getitem__(self, idx: int):
        with self.annotations_paths[idx].open('r') as f:
            for line in f.readlines():
                label, bbox = self.maybe_convert_polygon_to_bbox(line)
                return label, bbox 
            return  torch.tensor(IGNORE), torch.as_tensor([IGNORE, IGNORE, IGNORE, IGNORE], dtype=torch.float32)
        
    def __len__(self):
        return len(self.annotations_paths)

Let's try it out

In [ ]:
ds = AnnotationDataset(RF100_ROOT / df.index[0] / 'test' / 'labels')
ds[0]

gg. Now we can use a torch `DataLoader` to speed up stuff. Let's define a couple of functions to help us out

In [ ]:
def get_areas_and_labels(dataset: str, split: str ="test"):
    ds = AnnotationDataset(RF100_ROOT / dataset / split / 'labels')
    dl = DataLoader(ds, 
#                     num_workers=1, 
                    batch_size=128)

    all_areas = None
    all_labels = None
    for (labels, bboxes) in dl:
        bboxes = bboxes[labels != IGNORE] 
        # area = w * h
        areas = bboxes[:,2] * bboxes[:,3]
        all_areas = torch.cat((all_areas, areas)) if all_areas is not None else areas
        all_labels = torch.cat((all_labels, labels)) if all_labels is not None else labels

    return all_areas, all_labels


def compute_stats(areas: torch.Tensor):
    # let's compute the number of small, medium and large bbox
    bins = torch.histc(areas, bins=3, min=0, max=0.3)
    return areas.mean(), areas.std(), *bins

In [ ]:
@maybe_load_from_disk(Path("../temp/bbox.pkl"))
def create_bbox_df(df):
    records = []
    dataset_bar = tqdm(df.index)
    for dataset in dataset_bar:
        dataset_bar.set_postfix_str(dataset)
        split_bar = tqdm(["train", "test", "valid"], leave=False)
        for split in split_bar:
            split_bar.set_postfix_str(split)
            areas, labels = get_areas_and_labels(dataset, split)
            vals = compute_stats(areas)
            vals = [val.float().item() for val in vals]
            area_mean, area_std, num_small, num_medium, num_large = vals
            labels = labels[labels != IGNORE]
            records.append(dict(
                                num_classes=labels.unique().numpy().shape[0],
                                labels=labels.unique().numpy(),
                                areas=areas.numpy(),
                                area_mean=area_mean, 
                                area_std=area_std, 
                                num_small=num_small, 
                                num_medium=num_medium, 
                                num_large=num_large,
                                split=split,
                                dataset=dataset,
                            )
                          )

    return pd.DataFrame.from_records(records)

In [ ]:
bbox_df = create_bbox_df(df)
bbox_df

In [ ]:
train_df = bbox_df[bbox_df["split"] == "train"].reset_index(drop=True)
valid_df = bbox_df[bbox_df["split"] == "valid"].reset_index(drop=True)
test_df = bbox_df[bbox_df["split"] == "test"].reset_index(drop=True)

In [ ]:
train_df

check if we have all the labels in all splits

In [ ]:
import numpy as np 

all_missing_labels = []
all_is_correct = []
for idx, (train_labels, valid_labels, test_labels) in tqdm(
                                zip(train_df.index,
                                    zip(train_df["labels"].values,  valid_df["labels"].values,  test_df["labels"].values
                                       )
                                   ),
                                total=len(train_df)):
    # see https://numpy.org/doc/stable/reference/generated/numpy.setdiff1d.html
    missing_from_valid = np.setdiff1d(valid_labels, train_labels)
    missing_from_test = np.setdiff1d(test_labels, train_labels)
    missing_labels = np.array([])
    
    if missing_from_valid.shape[0] > 0:
        missing_labels = np.concatenate((missing_labels, missing_from_valid))
    if missing_from_test.shape[0] > 0:
        missing_labels = np.concatenate((missing_labels, missing_from_test))
    
    is_correct = missing_labels.shape[0] == 0
    all_missing_labels.append(missing_labels)
    all_is_correct.append(is_correct)

train_df["missing_labels"] = all_missing_labels
train_df["is_correct"] = all_is_correct


In [ ]:
train_df

In [ ]:
wrong_df = train_df[~train_df["is_correct"]]

In [ ]:
wrong_df[["dataset", "missing_labels"]].to_csv("missing.csv")

Let's add all the prev informations

merging areas

In [ ]:
records = {}

for idx, train_area, valid_area, test_area in zip(train_df["dataset"], 
                                                  train_df["areas"].values, 
                                                  valid_df["areas"].values,
                                                  test_df["areas"].values):
    records[idx] = np.concatenate([train_area, valid_area, test_area])
    

areas_series = pd.Series(records)
areas_series

In [ ]:
bbox_df_grouped["areas"] = areas_series
bbox_df_grouped.head(2)

In [ ]:
df_with_bbox = df.join(bbox_df_grouped, how="inner")

In [ ]:
df_with_bbox

In [ ]:
fig = plt.figure(figsize=(25, 40))
plot = sns.boxplot(data=df_with_bbox["areas"], orient='h')
plt.xlabel("bbox")
plt.ylabel("dataset")
plot.set_yticklabels(df.index)
plt.gcf().savefig("plot_all_train.png")

In [ ]:
plt.gcf().savefig("plot.png")

### Clip Embeddings

I have sampled 512 points per dataset and encoded them with CLIP. Let's load them, avg them and plot in 2D (after PCA). Let's do it! 

In [ ]:
import clip
from typing import Callable
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
RF100_ROOT = Path("./rf100/")

class ImageDataset(Dataset):
    def __init__(self, root: Path, fmt: str = "jpg", transform: Callable = None):
        super().__init__()
        self.images_path = list(root.glob(f"**/*.{fmt}"))
        self.transform = transform or ToTensor()

    def __getitem__(self, idx: int):
        image = Image.open(self.images_path[idx]).convert("RGB")
        return self.transform(image), idx, str(self.images_path[idx])

    def __len__(self):
        return len(self.images_path)

def pca(x, k, center=True):
    if center:
        m = x.mean(0, keepdim=True)
        s = x.std(0, unbiased=False, keepdim=True)
        x -= m
        x /= s
    # why pca related to svd? https://www.cs.cmu.edu/~elaw/papers/pca.pdf chap VI
    U, S, V = torch.linalg.svd(x) 
    reduced = torch.mm(x, V[:k].T)

    return reduced

@maybe_load_from_disk(Path("./embeddings_means.pkl"))
def get_embeddings(df):
    MAX_BATCHES = 2
    model, preprocess = clip.load("ViT-B/32", device=device, jit=True)
    records = []
    for dataset in tqdm(df.index):
        ds = ImageDataset(RF100_ROOT / dataset / "train/images", transform=preprocess)
        dl = DataLoader(
            ds, batch_size=256, num_workers=8, pin_memory=True, shuffle=True
        )  # we shuffle and we sample MAX_BATCHES batches per dataset
        i = 0
        means = None
        for (x, _, _) in dl:
            with torch.no_grad():
                x = x.to("cuda")
                x = model.encode_image(x)
                means = torch.vstack((means, x.mean(0))) if means is not None else x.mean(0)
            i += 1
            if i >= MAX_BATCHES: break
        if len(means.shape) == 1: means = means.unsqueeze(0)
        
        records.append(dict(dataset=dataset, clip_mean=means.mean(0).squeeze().cpu().numpy()))
            
    return pd.DataFrame.from_records(records, index=df.index)




In [ ]:
embed_df = get_embeddings(df)
embed_df.head(2)

In [ ]:
clip_means = torch.stack([torch.from_numpy(el) for el in embed_df.clip_mean.tolist()])

In [ ]:
clip_means_reduced = pca(clip_means.float(), k=2)
clip_means_reduced.shape

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = sns.scatterplot(x=clip_means_reduced[:,0], y=clip_means_reduced[:,1], size=df["size"], sizes=(0, 500), hue=df.category, alpha=.66, legend="brief")
sns.move_legend(ax, bbox_to_anchor=(1.02, 1), loc='upper left')

In [ ]:
plt.savefig("embedds.png",)

## Plotting

In [ ]:
df_with_bbox

In [ ]:
plt.savefig("datasets.png")

In [ ]:
fig, axs = plt.subplots(2, 2,  figsize=(20,16))
import matplotlib as mpl
mpl.rcParams['axes.titlepad'] = 16
# fig.suptitle('Datasets Categories')
axs[0,0].set_title("Total datasets size/category")
ax = sns.barplot(data=grouped_df, x="size", y="category", linewidth=2,  edgecolor=".2", ax=axs[0,0])
ax.set_ylabel('category')
ax.set_xlabel('size')
secax = ax.secondary_xaxis('top', functions=(lambda x: x / df_with_bbox["size"].sum(), lambda x:x))
secax.set_xlabel('size (%)')
ax.minorticks_on()
secax.minorticks_on()

axs[0,1].set_title("Mean datasets size/category")
ax = sns.boxplot(data=df_with_bbox, x="size", y="category", ax=axs[0,1])
ax.set_xlabel('size')
ax.get_yaxis().set_visible(False)
secax = ax.secondary_xaxis('top', functions=(lambda x: x / df_with_bbox["size"].sum(), lambda x:x))
secax.set_xlabel('size (%)')
ax.minorticks_on()
secax.minorticks_on()

axs[1,0].set_title("Mean bbox area")
ax = sns.boxplot(data=df_with_bbox, x="area_mean", y="category", ax=axs[1,0])
ax.set_xlabel("bbox")

axs[1,1].set_title("Mean num_classes")
ax = sns.boxplot(data=df_with_bbox, x="num_classes", y="category", ax=axs[1,1])
ax.set_xlabel("labels")
ax.get_yaxis().set_visible(False)


In [ ]:
plt.savefig("datasets_stats.png")